In [1]:
import pandas as pd
import duckdb

In [2]:
conn = duckdb.connect('r2_prata.duckdb')
conn.execute("ATTACH 'r2_ouro.duckdb' AS r2_ouro")

def sql(query):
    return conn.execute(query).fetch_df()

show_tables_prata_query = """ SELECT DISTINCT table_name FROM information_schema.tables WHERE table_catalog = 'r2_prata' """


show_tables_ouro_query = """ SELECT DISTINCT table_name FROM information_schema.tables WHERE table_catalog = 'r2_ouro' """


In [68]:
sql(show_tables_prata_query+" AND table_name LIKE 'fato_estacao%'")

,table_name
0,fato_estacao_CEMADEN
1,fato_estacao_UNESP
2,fato_estacao_INMET
3,fato_estacao_CEMIG
4,fato_estacao_IAC
5,fato_estacao_INMET_AGRI
6,fato_estacao_ANA


In [110]:
sql(show_tables_ouro_query)

,table_name
0,estacoes_distancias
1,fato_estacoes_precipitacao
2,fato_estacoes_temperatura_maxima
3,fato_estacoes_umidade_relativa_maxima
4,fato_estacoes_umidade_relativa_media
5,fato_estacoes_temperatura_minima
6,fato_estacoes_umidade_relativa_minima
7,dim_estacoes_escolha_vizinha
8,fato_estacoes_temperatura_media


# fato_estacoes_{column}

In [102]:
def construir_tabela_fato(estacoes,column):
    estacoes_selecionadas = {estacao:df for estacao,df in estacoes.items() if any(column in c for c in df.columns)}
    fato = None
    for estacao,df in estacoes_selecionadas.items():
        if fato is None:
            fato = df[['id_estacao','dt_medicao',column]].copy()
            fato['nm_grupo_estacao'] = estacao
        else:
            new_df = df[['id_estacao','dt_medicao',column]].copy()
            new_df['nm_grupo_estacao'] = estacao
            fato = pd.concat([fato,new_df])

    return fato

In [103]:
estacoes_list = sql(show_tables_prata_query+" AND table_name LIKE 'fato_estacao%'")['table_name'].to_list()
estacoes = {estacao.split("_")[-1]:sql(f"SELECT * FROM r2_prata.{estacao}") for estacao in estacoes_list}

In [104]:
fato_estacoes_temperatura_minima = construir_tabela_fato(estacoes,'vl_temperatura_minima')
fato_estacoes_temperatura_media = construir_tabela_fato(estacoes,'vl_temperatura_media')
fato_estacoes_temperatura_maxima = construir_tabela_fato(estacoes,'vl_temperatura_maxima')
fato_estacoes_umidade_relativa_minima = construir_tabela_fato(estacoes,'vl_umidade_relativa_minima')
fato_estacoes_umidade_relativa_media = construir_tabela_fato(estacoes,'vl_umidade_relativa_media')
fato_estacoes_umidade_relativa_maxima = construir_tabela_fato(estacoes,'vl_umidade_relativa_maxima')
fato_estacoes_precipitacao = construir_tabela_fato(estacoes,'vl_precipitacao')

In [109]:
columns = ['vl_temperatura_minima','vl_temperatura_media','vl_temperatura_maxima','vl_umidade_relativa_minima','vl_umidade_relativa_media','vl_umidade_relativa_maxima','vl_precipitacao']
for column in columns:
    sql(
    f"""
    CREATE TABLE IF NOT EXISTS r2_ouro.fato_estacoes_{column[3:]} AS SELECT * FROM fato_estacoes_{column[3:]}
    """
    )

# dim_estacoes_escolha_vizinha

In [6]:
sql(
f"""
CREATE TABLE IF NOT EXISTS r2_ouro.dim_estacoes_escolha_vizinha AS (
WITH enriquecimento AS (
    SELECT
        t1.id_estacao AS id_estacao_base
        ,t1.dt_medicao
        ,t2.id_estacao AS id_estacao_vizinha
        ,CASE
            WHEN t2.vl_precipitacao IS NULL THEN 0
            ELSE 1
        END AS fl_enriquecimento   

    FROM r2_ouro.fato_estacoes_precipitacao AS t1

    LEFT JOIN r2_ouro.fato_estacoes_precipitacao AS t2
        ON t1.dt_medicao = t2.dt_medicao
        AND t2.id_estacao <> t1.id_estacao

    WHERE 1=1
        AND t1.vl_precipitacao IS NOT NULL
        AND t2.id_estacao IS NOT NULL)

SELECT
    enriquecimento.id_estacao_base
    ,enriquecimento.id_estacao_vizinha
    ,1-SUM(enriquecimento.fl_enriquecimento)/COUNT(enriquecimento.fl_enriquecimento) AS pct_dados_faltantes_intersecao
    ,distancias.distancia_km
FROM enriquecimento

JOIN r2_ouro.estacoes_distancias AS distancias
    ON distancias.id_estacao_1 = enriquecimento.id_estacao_base
    AND distancias.id_estacao_2 = enriquecimento.id_estacao_vizinha

GROUP BY
    enriquecimento.id_estacao_base
    ,enriquecimento.id_estacao_vizinha
    ,distancias.distancia_km)
""")

,Count


In [111]:
conn.close()